In [1]:
import pandas as pd
import sqlite3
from data_packers import NFL_Data_Packer
from dbmanager import SqliteDBManager
import json
import copy
from datetime import datetime, timedelta
from dataclasses import dataclass

nfl_data_packer = NFL_Data_Packer("SportsData.db")
manager = SqliteDBManager("SportsData.db")

In [2]:
nfl_data_packer.pack_own_odds_predictions()

Records inserted successfully into STORED_GAME_OWN_ODDS
Table 'GAME_OWN_ODDS' truncated successfully.
Records inserted successfully into GAME_OWN_ODDS
Successfully packed game_own_odds_table


Create a query that will pull home team, away team, proj score home, proj score away, proj spread, proj total

In [ ]:
def get_predicted_info_by_team():
    manager.dataframe_query(
    """
    SELECT
        HOME_TEAM.TEAM_NAME AS HOME_TEAM,
        AWAY_TEAM.TEAM_NAME AS AWAY_TEAM,
        GOO.PREDICTED_HOME_POINTS AS PREDICTED_HOME_SCORE,
        GOO.PREDICTED_AWAY_POINTS AS PREDICTED_AWAY_SCORE,
        GOO.SPREAD AS HOME_SPREAD,
        GOO.TOTAL AS PREDICTED_TOTAL,
        GAMES.GAME_DATE
    FROM
        GAME_OWN_ODDS AS GOO JOIN GAMES AS GAMES ON GOO.GAME_SERIAL = GAMES.SERIAL
        JOIN TEAMS AS HOME_TEAM ON HOME_TEAM.SERIAL = GAMES.HOME_TEAM_SERIAL
        JOIN TEAMS AS AWAY_TEAM ON AWAY_TEAM.SERIAL = GAMES.AWAY_TEAM_SERIAL
    LIMIT 5
    """)

,HOME_TEAM,AWAY_TEAM,PREDICTED_HOME_SCORE,PREDICTED_AWAY_SCORE,HOME_SPREAD,PREDICTED_TOTAL,GAME_DATE
0,Green Bay Packers,Miami Dolphins,21.79,20.36,-1.43,42.15,2024-11-28 20:20:00
1,Kansas City Chiefs,Las Vegas Raiders,27.50,20.31,-7.19,47.81,2024-11-29 15:00:00
2,Minnesota Vikings,Arizona Cardinals,21.76,18.91,-2.85,40.67,2024-12-01 13:00:00
3,Atlanta Falcons,Los Angeles Chargers,19.07,24.56,5.49,43.63,2024-12-01 13:00:00
4,Cincinnati Bengals,Pittsburgh Steelers,24.77,26.22,1.45,50.99,2024-12-01 13:00:00


## Get own odds for each game.

In [ ]:
manager.check_table_info("GAME_OWN_ODDS")

[('SERIAL', 'INTEGER', 'PRIMARY KEY'),
 ('GAME_SERIAL', 'INTEGER', 'REFERENCES GAMES(SERIAL)'),
 ('SPREAD', 'FLOAT'),
 ('TOTAL', 'FLOAT'),
 ('PREDICTED_HOME_POINTS', 'FLOAT',),
 ('PREDICTED_AWAY_POINTS', 'FLOAT'),
 ('UPDATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP')]

[(0, 'SERIAL', 'INTEGER', 0, None, 1, 0),
 (1, 'GAME_SERIAL', 'INTEGER', 0, None, 0, 0),
 (2, 'SPREAD', 'FLOAT', 0, None, 0, 0),
 (3, 'TOTAL', 'FLOAT', 0, None, 0, 0),
 (4, 'PREDICTED_HOME_POINTS', 'FLOAT', 0, None, 0, 0),
 (5, 'PREDICTED_AWAY_POINTS', 'FLOAT', 0, None, 0, 0),
 (6, 'UPDATED_DATE', 'TIMESTAMP', 0, 'CURRENT_TIMESTAMP', 0, 0)]

In [4]:
manager.create_table('STORED_GAME_OWN_ODDS', [('SERIAL', 'INTEGER', 'PRIMARY KEY'),
 ('GAME_SERIAL', 'INTEGER', 'REFERENCES GAMES(SERIAL)'),
 ('SPREAD', 'FLOAT'),
 ('TOTAL', 'FLOAT'),
 ('PREDICTED_HOME_POINTS', 'FLOAT',),
 ('PREDICTED_AWAY_POINTS', 'FLOAT'),
 ('UPDATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP')])

Table 'STORED_GAME_OWN_ODDS' created successfully.


In [2]:
league_serial = 1

In [ ]:
# Should add a table to the db for this, truncate and make enw every time after adding ppg stats.



In [11]:
team_ppg_data = manager.dataframe_query(
f"""
SELECT
    TEAMS.SERIAL,
    PPG.THIS_YEAR_PPG AS PPG_2024,
    OPPG.THIS_YEAR_PPG AS OPPG_2024,
    PPG.LAST_3 AS PPG_LAST_3,
    OPPG.LAST_3 AS OPPG_LAST_3,
    PPG.HOME AS PPG_HOME,
    OPPG.HOME AS OPPG_HOME,
    PPG.AWAY AS PPG_AWAY,
    OPPG.AWAY AS OPPG_AWAY
FROM
    TEAMS AS TEAMS
    JOIN TEAM_TR_FOOTBALL_PPG AS PPG ON TEAMS.SERIAL = PPG.TEAM_SERIAL
    JOIN TEAM_TR_FOOTBALL_OPPG AS OPPG ON TEAMS.SERIAL = OPPG.TEAM_SERIAL
WHERE
    TEAMS.LEAGUE_SERIAL = 1 AND
    PPG.UPDATED_DATE = (SELECT MAX(UPDATED_DATE) FROM TEAM_TR_FOOTBALL_PPG)
    AND OPPG.UPDATED_DATE = (SELECT MAX(UPDATED_DATE) FROM TEAM_TR_FOOTBALL_OPPG)
ORDER BY
    TEAMS.TEAM_NAME
""")
@dataclass
class GameOwnOddsRow:
    game_serial: int
    predicted_home_points: float
    predicted_away_points: float

    @property
    def spread(self) -> float:
        return round(self.predicted_home_points - self.predicted_away_points,2)
    
    @property
    def total(self) -> float:
        return round(self.predicted_home_points + self.predicted_away_points,2)
    
    def to_dict(self):
        """
        Convert the values in the object to a dictionary, and enforce type security
        for the database
        """
        return {
            "game_serial": self.game_serial,
            "spread": float(self.spread),
            "total": float(self.total),
            "predicted_home_points": float(self.predicted_home_points),
            "predicted_away_points": float(self.predicted_away_points)   
        }

def get_weighted_average(stats, weights):
    """Calculate a weighted average."""
    return sum(stat * weight for stat, weight in zip(stats, weights))

def get_predicted_scores(home_serial: int = None, away_serial: int = None) -> list[float, float]:
    """
    Using the serials of two teams, get predicted scores for each team.
    ORDER MATTERS.

    Kwargs:
      home_serial (int): The serial of the home team.
      away_serial (int): the serial of the away team.

    Returns:
     A list with two floats that have the values: [home_prediction,away_prediction].
    """

    # Pre-filter the DataFrame for both teams
    filtered_data = team_ppg_data[team_ppg_data['SERIAL'].isin([away_serial, home_serial])]

    # Ensure data for both teams is found
    if filtered_data.shape[0] != 2:
        raise ValueError("One or both team SERIALs not found in the data.")

    # Split into home and away records
    away_record = filtered_data[filtered_data['SERIAL'] == away_serial].iloc[0]
    home_record = filtered_data[filtered_data['SERIAL'] == home_serial].iloc[0]

    # Define stats and weights
    away_stats = [away_record.PPG_2024, home_record.OPPG_2024, away_record.PPG_AWAY,
                home_record.OPPG_HOME, away_record.PPG_LAST_3, home_record.OPPG_LAST_3]
    home_stats = [home_record.PPG_2024, away_record.OPPG_2024, home_record.PPG_HOME,
                away_record.OPPG_AWAY, home_record.PPG_LAST_3, away_record.OPPG_LAST_3]
    weights = [0.1, 0.1, 0.2, 0.2, 0.2, 0.2]

    # Calculate weighted averages
    away_team_score_prediction = round(get_weighted_average(away_stats, weights),2)
    home_team_score_prediction = round(get_weighted_average(home_stats, weights),2)

    return [home_team_score_prediction, away_team_score_prediction]


    
def get_predictions_for_games():
    games_get_odds = manager.dataframe_query(
    f"""
    SELECT *
    FROM GAMES
    WHERE 
    LEAGUE_SERIAL = 1 AND
    GAME_DATE > '{datetime.now().strftime(r"%Y-%m-%d %H:%M:%S")}'
    """)

    game_own_odds_rows = []

    for row in games_get_odds.itertuples():
        predicted_home_points, predicted_away_points = get_predicted_scores(home_serial=row.HOME_TEAM_SERIAL,
                                                                            away_serial=row.AWAY_TEAM_SERIAL)
        
        game_own_odds_rows.append(GameOwnOddsRow(row.SERIAL, predicted_home_points, predicted_away_points))

    rows_as_tuples = [tuple(row.to_dict().values()) for row in game_own_odds_rows]

    return rows_as_tuples

In [12]:
get_predictions_for_games()

[(177, 1.43, 42.15, 21.79, 20.36),
 (178, 7.19, 47.81, 27.5, 20.31),
 (179, 2.85, 40.67, 21.76, 18.91),
 (180, -5.49, 43.63, 19.07, 24.56),
 (181, -1.45, 50.99, 24.77, 26.22),
 (182, -6.73, 46.37, 19.82, 26.55),
 (183, 0.05, 43.31, 21.68, 21.63),
 (184, -1.73, 40.99, 19.63, 21.36),
 (185, 4.67, 48.73, 26.7, 22.03),
 (186, -6.22, 49.06, 21.42, 27.64),
 (187, 4.83, 46.87, 25.85, 21.02),
 (188, -3.3, 49.62, 23.16, 26.46),
 (189, 8.32, 49.18, 28.75, 20.43),
 (190, 9.84, 40.54, 25.19, 15.35),
 (191, 7.38, 47.98, 27.68, 20.3),
 (192, 6.49, 43.39, 24.94, 18.45),
 (193, 9.57, 46.31, 27.94, 18.37),
 (194, 6.78, 41.34, 24.06, 17.28),
 (195, 4.4, 44.74, 24.57, 20.17),
 (196, 7.01, 49.95, 28.48, 21.47),
 (197, 6.07, 44.07, 25.07, 19.0),
 (198, -7.09, 41.69, 17.3, 24.39),
 (199, 2.71, 40.49, 21.6, 18.89),
 (200, -5.37, 47.59, 21.11, 26.48),
 (201, 0.78, 42.0, 21.39, 20.61),
 (202, -1.33, 43.49, 21.08, 22.41),
 (203, -9.74, 53.06, 21.66, 31.4)]

[(1, 24.75, 18.01, 6.74, 42.76),
 (2, 18.6, 23.42, -4.82, 42.02),
 (3, 25.03, 20.73, 4.3, 45.76),
 (4, 20.63, 19.46, 1.17, 40.09),
 (5, 22.15, 16.4, 5.75, 38.55)]

In [20]:
tuple(game_own_odds_rows[0].to_dict().values())

(1, 24.75, 18.01, 6.74, 42.76)

In [13]:
game_own_odds_rows[0].__dict__

{'game_serial': 1,
 'predicted_home_points': np.float64(24.75),
 'predicted_away_points': np.float64(18.01)}

To get  Away team stats you take:
    away team ppg for this year * .10 +
    the home teams oppg * .10 +
    away teams ppg for away games * .20 +
    home teams oppg for home games * .20 +
    away team ppg for last 3 games * .20 +
    home teams ppg for last 3 games * .20

### Test others

In [ ]:
# Define the file path and name
file_path = "sample_odds_api_call_11-19-24.json"

# Save the JSON object to a file
# with open(file_path, "w") as json_file:
#     json.dump(odds_json, json_file, indent=4)  # Use indent for pretty formatting

# Open and load the JSON file
with open(file_path, "r") as json_file:
    response_data = json.load(json_file)
markets = nfl_data_packer.get_market_info()
market_dfs = []

for serial, market_name in markets:
    market_dfs.append(nfl_data_packer.parse_market_info(market_name, serial, response_data))
nfl_data_packer.odds_df= pd.concat(market_dfs, axis=0)

nfl_data_packer.odds_df

In [12]:
odds_df = pd.read_csv("odds_df_11-19-24.csv")
odds_df = odds_df.drop(['Unnamed: 0',], axis=1)

In [19]:
nfl_data_packer.odds_df = odds_df

nfl_data_packer.odds_df

,Home_Team,Away_Team,Date,MARKET_SERIAL,H2H_HOME,H2H_AWAY,SPREAD_HOME,TOTAL,home_serial,away_serial
0,Cleveland Browns,Pittsburgh Steelers,2024-11-21 20:15:00,1,2.64,1.51,3.5,36.5,8,27
1,Carolina Panthers,Kansas City Chiefs,2024-11-24 13:00:00,1,5.70,1.15,11.0,42.5,5,16
2,Chicago Bears,Minnesota Vikings,2024-11-24 13:00:00,1,2.45,1.57,3.5,39.5,6,21
3,Washington Commanders,Dallas Cowboys,2024-11-24 13:00:00,1,5.25,1.17,10.5,45.5,32,9
4,Indianapolis Colts,Detroit Lions,2024-11-24 13:00:00,1,1.24,4.30,-7.5,50.5,14,11
5,Houston Texans,Tennessee Titans,2024-11-24 13:00:00,1,1.23,4.40,-8.5,42.0,13,31
6,Miami Dolphins,New England Patriots,2024-11-24 13:00:00,1,1.28,3.85,-7.0,46.0,17,22
7,New York Giants,Tampa Bay Buccaneers,2024-11-24 13:00:00,1,3.15,1.38,5.5,42.0,24,30
8,Las Vegas Raiders,Denver Broncos,2024-11-24 16:05:00,1,1.42,2.95,-5.5,41.0,18,10
9,Seattle Seahawks,Arizona Cardinals,2024-11-24 16:25:00,1,1.93,1.89,1.0,47.5,29,1


In [25]:
future_games = nfl_data_packer.populate_future_game_data()

future_games = future_games[['home_serial', 'away_serial', 'Date', 'league_serial']]

future_games

c:\Users\taylor.mccoy\OneDrive\Documents\Coding\Sports Data\data_packers.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmade_games['league_serial'] = self.league_serial


,home_serial,away_serial,Date,league_serial
0,8,27,2024-11-21 20:15:00,1
1,5,16,2024-11-24 13:00:00,1
2,6,21,2024-11-24 13:00:00,1
3,32,9,2024-11-24 13:00:00,1
4,14,11,2024-11-24 13:00:00,1
5,13,31,2024-11-24 13:00:00,1
6,17,22,2024-11-24 13:00:00,1
7,24,30,2024-11-24 13:00:00,1
8,18,10,2024-11-24 16:05:00,1
9,29,1,2024-11-24 16:25:00,1


In [26]:
nfl_data_packer.df_to_db('GAMES',future_games)

Records inserted successfully into GAMES


In [27]:
nfl_data_packer._combine_odds_api_and_games_dfs()

nfl_data_packer.combined_market_odds_df

,SERIAL,MARKET_SERIAL,SPREAD_HOME,TOTAL,H2H_HOME,H2H_AWAY
0,162,1,3.5,36.5,2.64,1.51
1,163,1,11.0,42.5,5.70,1.15
2,164,1,3.5,39.5,2.45,1.57
3,165,1,10.5,45.5,5.25,1.17
4,166,1,-7.5,50.5,1.24,4.30
5,167,1,-8.5,42.0,1.23,4.40
6,168,1,-7.0,46.0,1.28,3.85
7,169,1,5.5,42.0,3.15,1.38
8,170,1,-5.5,41.0,1.42,2.95
9,171,1,1.0,47.5,1.93,1.89


In [28]:
nfl_data_packer.df_to_db('GAME_MARKET_ODDS', nfl_data_packer.combined_market_odds_df)

Records inserted successfully into GAME_MARKET_ODDS


In [11]:
game_serials = nfl_data_packer.combined_market_odds_df.SERIAL.unique()

combined_existing_game_data = nfl_data_packer.manager.dataframe_query(f"""SELECT * FROM GAME_MARKET_ODDS 
                                        WHERE GAME_SERIAL IN 
                                        {tuple(str(serial) for serial in game_serials)}""")
combined_existing_game_data.head()

,SERIAL,GAME_SERIAL,MARKET_SERIAL,SPREAD,TOTAL,H2H_HOME,H2H_AWAY,UPDATED_DATE
0,1,162,1,3.5,36.5,2.64,1.51,2024-11-20 02:55:04
1,2,163,1,11.0,42.5,5.70,1.15,2024-11-20 02:55:04
2,3,164,1,3.5,39.5,2.45,1.57,2024-11-20 02:55:04
3,4,165,1,10.5,45.5,5.25,1.17,2024-11-20 02:55:04
4,5,166,1,-7.5,50.5,1.24,4.30,2024-11-20 02:55:04


In [12]:
import copy

api_data = copy.deepcopy(nfl_data_packer.combined_market_odds_df)

In [29]:
combined_existing_game_data.head()

,SERIAL,GAME_SERIAL,MARKET_SERIAL,SPREAD,TOTAL,H2H_HOME,H2H_AWAY,UPDATED_DATE
0,1,162,1,3.5,28.5,2.64,1.51,2024-11-20 02:55:04
1,2,163,1,11.0,42.5,5.70,1.15,2024-11-20 02:55:04
2,3,164,1,3.5,39.5,2.45,1.57,2024-11-20 02:55:04
3,4,165,1,10.5,45.5,5.25,1.17,2024-11-20 02:55:04
4,5,166,1,-7.5,50.5,1.24,4.30,2024-11-20 02:55:04


In [28]:
combined_existing_game_data.iloc[0,4] = 28.5

In [31]:
# Function to compare row values
def has_changes(api_row, existing_row):
    return (
        api_row['SPREAD_HOME'] != existing_row.SPREAD or
        api_row['TOTAL'] != existing_row.TOTAL or
        api_row['H2H_HOME'] != existing_row.H2H_HOME or
        api_row['H2H_AWAY'] != existing_row.H2H_AWAY
    )

changed_rows = []
# for each existing row look for a matching api row and then append to new df
for row in combined_existing_game_data.itertuples():
    
    # Find matching rows in api_data
    api_data_match = api_data.loc[
        (api_data['SERIAL'] == int(row.GAME_SERIAL)) &
        (api_data['MARKET_SERIAL'] == int(row.MARKET_SERIAL))
    ]
    
    if api_data_match.empty:
        continue  # Skip if no match found
    
    # Handle single or multiple matches
    for _, api_row in api_data_match.iterrows():
        if has_changes(api_row, row):
            # Append tuple or other logic for changed rows
            changed_rows.append({
                'GAME_SERIAL': row.GAME_SERIAL,
                'MARKET_SERIAL': row.MARKET_SERIAL,
                'SPREAD_HOME': api_row['SPREAD_HOME'],
                'TOTAL': api_row['TOTAL'],
                'H2H_HOME': api_row['H2H_HOME'],
                'H2H_AWAY': api_row['H2H_AWAY']
            })

# Create DataFrame for new entries
new_entries = pd.DataFrame(changed_rows)
new_entries

,GAME_SERIAL,MARKET_SERIAL,SPREAD_HOME,TOTAL,H2H_HOME,H2H_AWAY
0,162,1,3.5,36.5,2.64,1.51


In [34]:
games_no_odds = manager.dataframe_query("""
                        SELECT 
                            G.GAME_DATE AS GAME_DATE, 
                            G.SERIAL AS GAME_SERIAL
                        FROM 
                            GAMES AS G
                        WHERE 
                            NOT EXISTS (
                                SELECT 1
                                FROM GAME_MARKET_ODDS AS GMO
                                WHERE GMO.GAME_SERIAL = G.SERIAL
                            )
                        """)

games_no_odds

,GAME_DATE,GAME_SERIAL
0,2024-09-08 00:00:00,1
1,2024-09-23 00:00:00,2
2,2024-10-20 00:00:00,3
3,2024-11-03 00:00:00,4
4,2024-09-15 00:00:00,5
...,...,...
156,2024-11-17 00:00:00,157
157,2024-11-17 00:00:00,158
158,2024-11-17 00:00:00,159
159,2024-11-17 00:00:00,160


In [33]:
most_recent_game_date_has_odds = manager.fetch_records("""
                        SELECT 
                            G.GAME_DATE AS GAME_DATE
                        FROM 
                            GAMES AS G JOIN GAME_MARKET_ODDS AS GMO ON G.SERIAL = GMO.GAME_SERIAL
                        WHERE 
                            G.GAME_DATE = (SELECT MAX(GAME_DATE) FROM GAMES)
                        """, fetchstyle='one')[0][0]

most_recent_game_date_odds_pulled

'2024-12-02 19:15:00'

In [38]:
odds_to_add = games_no_odds[(games_no_odds['GAME_DATE'] > most_recent_game_date_odds_pulled)]
odds_to_add

,GAME_DATE,GAME_SERIAL


In [27]:
nfl_data_packer.df_to_db('GAME_MARKET_ODDS', nfl_data_packer.combined_market_odds_df)

Records inserted successfully into GAME_MARKET_ODDS


In [3]:
nfl_data_packer.show_api_requests_used()


You have used 57 calls and have 443 remaining.


In [6]:
nfl_data_packer.pack_odds_df() # fix by adding home/away serial redoing df struct

ValueError: Number of columns in table and length of the first record tuple do not match

In [7]:
nfl_data_packer.populate_future_game_data()

Records inserted successfully into GAMES


In [10]:
import copy

changed_odds_check: pd.DataFrame = copy.deepcopy(nfl_data_packer.odds_df)

changed_odds_check['home_serial'] = changed_odds_check['Home_Team'].apply(nfl_data_packer.get_team_serial)
changed_odds_check['away_serial'] = changed_odds_check['Away_Team'].apply(nfl_data_packer.get_team_serial)

changed_odds_check['UID'] = changed_odds_check['UID'] = (
    changed_odds_check['home_serial'].astype(str) + ' ' +
    changed_odds_check['away_serial'].astype(str) + ' ' +
    changed_odds_check['Date'].astype(str)
)

games_df = nfl_data_packer.manager.dataframe_query('select * from GAMES')
games_df['UID'] = games_df['UID'] = (
    games_df['HOME_TEAM_SERIAL'].astype(str) + ' ' +
    games_df['AWAY_TEAM_SERIAL'].astype(str) + ' ' +
    games_df['GAME_DATE'].astype(str)
)

# Join the team df to the current db games data using the UID 
joined_df = pd.merge(changed_odds_check,games_df, on='UID', how='inner')

In [20]:
print(len(changed_odds_check))

26


In [21]:
print(len(joined_df.drop_duplicates(subset=['UID','MARKET_SERIAL'])))

26


In [8]:
nfl_data_packer.get_changed_market_odds()

KeyError: "['SERIAL'] not in index"

In [2]:
nfl_data_packer.db_last_updated_date

'2024-11-14 01:00:00'

In [3]:
nfl_data_packer.get_all_games_data() 

In [4]:
data = nfl_data_packer.all_games_data

In [5]:
for df in data.values():
    print(df['Date'])

10    2024-11-17 00:00:00
Name: Date, dtype: object
9    2024-11-17 00:00:00
Name: Date, dtype: object
9    2024-11-18 00:00:00
Name: Date, dtype: object
10    2024-11-17 00:00:00
Name: Date, dtype: object
9    2024-11-17 00:00:00
Name: Date, dtype: object
9    2024-11-17 00:00:00
Name: Date, dtype: object
9    2024-11-17 00:00:00
Name: Date, dtype: object
10    2024-11-17 00:00:00
Name: Date, dtype: object
10    2024-11-17 00:00:00
Name: Date, dtype: object
10    2024-11-17 00:00:00
Name: Date, dtype: object
9    2024-11-17 00:00:00
Name: Date, dtype: object
9    2024-11-17 00:00:00
Name: Date, dtype: object
9    2024-11-17 00:00:00
Name: Date, dtype: object


In [6]:
data.keys()

dict_keys(['Buffalo Bills', 'Chicago Bears', 'Dallas Cowboys', 'Denver Broncos', 'Detroit Lions', 'Miami Dolphins', 'Los Angeles Chargers', 'New England Patriots', 'New Orleans Saints', 'New York Jets', 'Pittsburgh Steelers', 'San Francisco 49ers', 'Tennessee Titans'])

In [7]:
# nfl_data_packer.pack_games_data()

Records inserted successfully into GAMES
Records inserted successfully into GAMES
Records inserted successfully into GAMES
Records inserted successfully into GAMES
Records inserted successfully into GAMES
Records inserted successfully into GAMES
Records inserted successfully into GAMES
Records inserted successfully into GAMES
Records inserted successfully into GAMES
Records inserted successfully into GAMES
Records inserted successfully into GAMES
Records inserted successfully into GAMES
Records inserted successfully into GAMES


In [8]:
# nfl_data_packer.pack_game_outcomes_data()

Records inserted successfully into GAME_OUTCOMES
Records inserted successfully into GAME_OUTCOMES
Records inserted successfully into GAME_OUTCOMES
Records inserted successfully into GAME_OUTCOMES
Records inserted successfully into GAME_OUTCOMES
Records inserted successfully into GAME_OUTCOMES
Records inserted successfully into GAME_OUTCOMES
Records inserted successfully into GAME_OUTCOMES
Records inserted successfully into GAME_OUTCOMES
Records inserted successfully into GAME_OUTCOMES
Records inserted successfully into GAME_OUTCOMES
Records inserted successfully into GAME_OUTCOMES
Records inserted successfully into GAME_OUTCOMES


c:\Users\taylor.mccoy\OneDrive\Documents\Coding\general_use_venv\Restricted\Sports Data\data_packers.py:391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['UID'] = (
c:\Users\taylor.mccoy\OneDrive\Documents\Coding\general_use_venv\Restricted\Sports Data\data_packers.py:391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['UID'] = (
c:\Users\taylor.mccoy\OneDrive\Documents\Coding\general_use_venv\Restricted\Sports Data\data_packers.py:391: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# test pack oppg pgg

nfl_data_packer.pack_ppg_oppg()

Records inserted successfully into TEAM_TR_FOOTBALL_PPG
Records inserted successfully into TEAM_TR_FOOTBALL_OPPG


In [6]:
manager.insert_table_records('MARKETS',[('DraftKings',),('FanDuel',)])

Records inserted successfully into MARKETS


In [13]:
manager.check_table_info('OWN_PPG_STATS')

[(0, 'SERIAL', 'INTEGER', 0, None, 1, 0), (1, 'TEAM_SERIAL', 'INTEGER', 0, None, 0, 0), (2, 'RANK', 'INTEGER', 0, None, 0, 0), (3, 'THIS_YEAR_PPG', 'INTEGER', 0, None, 0, 0), (4, 'LAST_3', 'INTEGER', 0, None, 0, 0), (5, 'LAST_1', 'INTEGER', 0, None, 0, 0), (6, 'HOME', 'INTEGER', 0, None, 0, 0), (7, 'AWAY', 'INTEGER', 0, None, 0, 0), (8, 'LAST_YEAR_PPG', 'INTEGER', 0, None, 0, 0), (9, 'UPDATED_DATE', 'TIMESTAMP', 0, 'CURRENT_TIMESTAMP', 0, 0)]


In [15]:
manager.create_table('OPPG',[
    ('SERIAL', 'INTEGER', 'PRIMARY KEY'),
    ('TEAM_SERIAL', 'INTEGER', 'NOT NULL REFERENCES TEAMS(SERIAL)'),  # Foreign key, not nullable
    ('RANK', 'INTEGER'),
    ('THIS_YEAR_PPG', 'INTEGER'),
    ('LAST_3', 'INTEGER'),
    ('LAST_1', 'INTEGER'),
    ('HOME', 'INTEGER'),
    ('AWAY', 'INTEGER'),
    ('LAST_YEAR_PPG', 'INTEGER'),
    ('UPDATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP')
])

Table 'OPPG' created successfully.


In [17]:
manager.drop_table('OWN_PPG_STATS')
manager.drop_table('OPPONENT_PPG_STATS')

Table 'OWN_PPG_STATS' deleted successfully.
Table 'OPPONENT_PPG_STATS' deleted successfully.


In [18]:
manager.alter_table('OPPG', operation='rename_table', new_name='TEAM_TR_FOOTBALL_OPPG')
manager.alter_table('PPG', operation='rename_table', new_name='TEAM_TR_FOOTBALL_PPG')


Table 'OPPG' altered successfully with operation 'rename_table'.
Table 'PPG' altered successfully with operation 'rename_table'.


In [3]:
# test odds api call

nfl_data_packer.get_odds_df()
nfl_data_packer.odds_df

,Home_Team,Away_Team,Date,DK_H2H_HOME,DK_H2H_AWAY,DK_SPREAD_HOME,DK_TOTAL,FD_H2H_HOME,FD_H2H_AWAY,FD_SPREAD_HOME,FD_TOTAL
0,Carolina Panthers,New York Giants,2024-11-10T14:30:00Z,3.30,1.35,6.5,40.5,3.20,1.37,6.5,40.5
1,New Orleans Saints,Atlanta Falcons,2024-11-10T18:00:00Z,1.54,2.54,-3.5,46.0,1.51,2.64,-3.5,46.5
2,Indianapolis Colts,Buffalo Bills,2024-11-10T18:00:00Z,1.51,2.64,-4.0,46.5,1.48,2.76,-3.5,46.5
3,Chicago Bears,New England Patriots,2024-11-10T18:00:00Z,1.35,3.30,-6.0,38.5,1.36,3.25,-6.5,38.5
4,Kansas City Chiefs,Denver Broncos,2024-11-10T18:00:00Z,4.00,1.26,7.5,42.0,4.15,1.25,7.5,41.5
5,Jacksonville Jaguars,Minnesota Vikings,2024-11-10T18:00:00Z,3.50,1.32,6.5,43.0,3.45,1.33,6.5,43.5
6,Washington Commanders,Pittsburgh Steelers,2024-11-10T18:00:00Z,2.24,1.68,3.0,45.0,2.26,1.68,2.5,44.5
7,Tampa Bay Buccaneers,San Francisco 49ers,2024-11-10T18:00:00Z,1.34,3.35,-6.5,50.0,1.36,3.30,-6.5,50.5
8,Los Angeles Chargers,Tennessee Titans,2024-11-10T21:05:00Z,1.31,3.60,-7.0,39.0,1.29,3.75,-7.5,38.5
9,Arizona Cardinals,New York Jets,2024-11-10T21:25:00Z,2.10,1.77,2.0,46.5,2.08,1.79,1.5,46.5


In [36]:
def cigar_party(cigars, is_weekend = False):
  if is_weekend:
    return (cigars >= 40)
  else:
    return (cigars >= 40 and cigars <= 60)

cigar_party(45)

True

In [1]:
[x for x in range(0,121) if (x % 10 >= 8 or x % 10 <= 2)]

[0,
 1,
 2,
 8,
 9,
 10,
 11,
 12,
 18,
 19,
 20,
 21,
 22,
 28,
 29,
 30,
 31,
 32,
 38,
 39,
 40,
 41,
 42,
 48,
 49,
 50,
 51,
 52,
 58,
 59,
 60,
 61,
 62,
 68,
 69,
 70,
 71,
 72,
 78,
 79,
 80,
 81,
 82,
 88,
 89,
 90,
 91,
 92,
 98,
 99,
 100,
 101,
 102,
 108,
 109,
 110,
 111,
 112,
 118,
 119,
 120]

In [33]:
near_ten(118)

True